In [2]:
# basic test
import numpy as np
from scipy.spatial import distance

def cal_cosine_distance(x, y):
    dist = distance.cosine(x, y)
    return dist

a = np.array([1, 1, 2, 3, 4])
b = np.array([1, 2, 2, 3, 4])

c = a.dot(b)/(np.linalg.norm(a) * np.linalg.norm(b)) # higher -> more similar
d = cal_cosine_distance(a, b) # smaller -> more similar

print(c)
print(d) 

0.985665765813916
0.014334234186084038


# prepared functions

In [3]:

import os
import pandas as pd
import copy
import numpy as np
from scipy.spatial import distance
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# 
def cal_cosine_distance(x, y):
    # The smaller value, the more similar for x and y
    dist = distance.cosine(x, y)
    return dist

def cal_cosine_similarity(x, y):
    # The bigger value, the more similar for x and y
    sim = 1 - distance.cosine(x, y)
    return sim

def get_img_array(path, show=False):
    tmp_img = Image.open(path)
    # reshape to (320, 240)
    tmp_img_reshape = tmp_img.resize((320, 240))
    
    if show:
        # Display the image
        plt.imshow(tmp_img_reshape)
        plt.axis('off')  # Turn off axis numbers
        plt.show()
    
    # convert the images to (R,G,B) arrays
    tmp_img_array = np.array(tmp_img_reshape)
    
    # flatten the arrays so they are 1 dimensional vectors
    tmp_img_array = tmp_img_array.flatten()
    
    # divide the arrays by 255, the maximum RGB value to make sure every value is on a 0-1 scale
    tmp_img_array = tmp_img_array/255
    
    return tmp_img_array

# rescale the distance value
def rescale(result):
    test = copy.deepcopy(result)
    sorted_test = sorted(test.items(), key=lambda item: item[1][2], reverse = True)
    print('max is:', sorted_test[0][1][2], '\nmin is:', sorted_test[-1][1][2])
    max_v = sorted_test[0][1][2]
    min_v = sorted_test[-1][1][2]
    print('rescaling...')
#     print(len(sorted_test))
    for i in range(len(sorted_test)):
        sorted_test[i][1][2] = (sorted_test[i][1][2] - min_v) / (max_v - min_v)
    print('rescaled.')
    return sorted_test

def get_target(emotion_name):
    # Anger, Disgust, Fear, Happiness, Sadness, Surprise, Neutral
    # or lowercase
    if emotion_name in ["Anger", "anger"]:
        return 0
    elif emotion_name in ["Disgust", "disgust"]:
        return 1
    elif emotion_name in ["Fear", "fear"]:
        return 2
    elif emotion_name in ["Happiness", "happiness"]:
        return 3
    elif emotion_name in ["Sadness", "sadness"]:
        return 4
    elif emotion_name in ["Surprise", "surprise"]:
        return 5
    elif emotion_name in ["Neutral", "neutral"]:
        return 6


# main loop for dealing with cosine distance 

In [147]:
!python --version
!pip list | grep feat

Python 3.8.17
py-feat                       0.6.1


In [5]:
a = 'image_analysis/Exp_collections/211223ExpHumanoidRaw/Exp7HumanoidRaw/anger/2021_12_23_15_32_24_anger_792_emotion.csv'
a = 'image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_39_37_anger_0_emotion.csv'
s = pd.read_csv(a)
print(s)

b = 'image_analysis/231219Exp16Feat037/anger/2023_12_19_14_49_07_anger_0_emotion.csv'
ss = pd.read_csv(b)
print(ss)

   Unnamed: 0     anger   disgust      fear  happiness   sadness  surprise  \
0           0  0.004275  0.001697  0.015044    0.09833  0.014817  0.723541   

    neutral                                              input  
0  0.142297  image_analysis/anger/2023_12_19_11_39_37_anger...  
   Unnamed: 0     anger   disgust      fear  happiness   sadness  surprise  \
0           0  0.007325  0.000414  0.004266   0.021711  0.003262  0.842857   

    neutral                                              input  
0  0.120165  image_analysis/anger/2023_12_19_14_49_07_anger...  


# new Py-Feat anylyze again

In [6]:
from feat import Detector

detector = Detector(emotion_model = "resmasknet", landmark_model='mobilefacenet')



In [24]:
import copy
import os

ALL_dic = {}
# for target_emotion in ['anger']:
for target_emotion in ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']:
    result_dic = {}
    folderName = os.path.join('image_analysis/231218Exp15NewFeat/', target_emotion)
    ls_name = os.listdir(folderName)
    for i in ls_name:
        if 'emotion' in i:
            mypath = os.path.join(folderName, i)
            
            # feat analyze
#             df = detector.detect_image(mypath)
#             result = df[target_emotion][0] 
            
            # csv analyze
            temp = pd.read_csv(mypath)
            result = temp[target_emotion][0]
            result = round(result, 6)
            fileName = temp['input'][0]
            result_dic[os.path.join(folderName, i[:-12] + '.png')] = result
    print('new feat emo finished')
    

    folderName = os.path.join('image_analysis/231219Exp16Feat037/', target_emotion)
    ls_name = os.listdir(folderName)
    count = 0
    for i in ls_name:
        if 'png' in i:
            mypath = os.path.join(folderName, i)
            
            # feat analyze
            df = detector.detect_image(mypath)
            result = df[target_emotion][0] 
            
            # csv analyze
#             temp = pd.read_csv(mypath)
#             result = temp[target_emotion][0]
            result = round(result, 6)
            fileName = df['input'][0]
            result_dic[fileName] = result
            
#             count += 1
#             if count > 10:
#                 break
    print('old ver image new feat analyze finished')
    ALL_dic[target_emotion] = copy.deepcopy(result_dic)
    

new feat emo finished


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/it]


old ver image new feat analyze finished
new feat emo finished


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.29s/it]


old ver image new feat analyze finished
new feat emo finished


...

old ver image new feat analyze finished
new feat emo finished


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.26s/it]


old ver image new feat analyze finished
new feat emo finished


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/it]

old ver image new feat analyze finished


In [35]:
print(ALL_dic['happiness'])

{'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_17_13_happiness_382.png': 0.002339, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_12_58_56_happiness_46.png': 0.171842, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_01_29_happiness_94.png': 0.118646, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_09_20_happiness_240.png': 0.040272, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_15_26_happiness_350.png': 0.764839, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_13_13_happiness_311.png': 0.901673, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_11_29_happiness_279.png': 0.144624, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_12_59_27_happiness_56.png': 0.665028, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_01_59_happiness_104.png': 0.089399, 'image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_19_08_happiness_415.png': 0.449438, 'image_analysis/231218Exp15NewFeat/happiness/2023_12

In [21]:
import copy
import os
for target_emotion in ['anger']:
    
    result_dic = {}
    
    folderName = os.path.join('image_analysis/231218Exp15NewFeat/', target_emotion)
    ls_name = os.listdir(folderName)
    for i in ls_name:
        if 'emotion' in i:
            mypath = os.path.join(folderName, i)
            
            # feat analyze
#             df = detector.detect_image(mypath)
#             result = df[target_emotion][0] 
            
            # csv analyze
            temp = pd.read_csv(mypath)
            result = temp[target_emotion][0]
            result = round(result, 6)
            fileName = temp['input'][0]
            result_dic[os.path.join(folderName, i[:-12] + '.png')] = result
    print('new feat emo finished')
    

    folderName = os.path.join('image_analysis/231219Exp16Feat037/', target_emotion)
    ls_name = os.listdir(folderName)
    count = 0
    for i in ls_name:
        if 'png' in i:
            mypath = os.path.join(folderName, i)
            
            # feat analyze
            df = detector.detect_image(mypath)
            result = df[target_emotion][0] 
            
            # csv analyze
#             temp = pd.read_csv(mypath)
#             result = temp[target_emotion][0]
            result = round(result, 6)
            fileName = df['input'][0]
            result_dic[fileName] = result
            
            count += 1
            if count > 10:
                break
    print('old ver image new feat analyze finished')
    

{'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_10_59_anger_474.png': 0.013084,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_04_21_anger_347.png': 0.135225,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_02_27_anger_308.png': 0.449606,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_03_30_anger_330.png': 0.026985,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_06_16_anger_386.png': 0.048187,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_01_56_anger_298.png': 0.05281,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_06_26_anger_389.png': 0.467451,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_08_57_anger_437.png': 0.273106,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_14_54_37_anger_138.png': 0.021715,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_02_59_anger_319.png': 0.370781,
 'image_analysis/231219Exp16Feat037/anger/2023_12_19_15_11_17_anger_480.png': 0.003828}

# choose top 10

## ALL_dic, top 50

In [149]:
for target_emotion in ['fear']:
    res_value_order = sorted(ALL_dic[target_emotion].items(), key=lambda item: item[1], reverse = True)
    res_value_order = [list(x) for x in res_value_order[:100]]
#     print(target_emotion)
    for i in res_value_order:
            print(i)
#     print(res_value_order)
#     print()


# len(ALL_dic[target_emotion])

['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642]
['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_22_fear_321.png', 0.930199]
['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_54_17_fear_334.png', 0.92667]
['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_50_30_fear_242.png', 0.921012]
['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_41_58_fear_32.png', 0.918868]
['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_40_52_fear_218.png', 0.914898]
['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_54_23_fear_462.png', 0.911952]
['image_analysis/231219Exp16Feat037/fear/2023_12_19_17_01_10_fear_491.png', 0.911653]
['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_37_02_fear_146.png', 0.910754]
['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_50_12_fear_235.png', 0.910513]
['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_44_44_fear_102.png', 0.90884]
['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46

In [64]:
angry_temp = [['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_07_32_anger_410.png', 0.695808], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_56_09_anger_356.png', 0.666345], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_00_06_anger_257.png', 0.630666], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_06_anger_53.png', 0.611739], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_59_43_anger_249.png', 0.607058], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_57_53_anger_391.png', 0.600057], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_00_17_anger_261.png', 0.593918], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_49_50_anger_224.png', 0.58142], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_54_anger_72.png', 0.568008], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_03_50_anger_337.png', 0.564389], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_50_38_anger_41.png', 0.560278], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_54_06_anger_314.png', 0.545188], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_03_59_anger_340.png', 0.539428], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_46_anger_69.png', 0.536772], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_07_26_anger_408.png', 0.534337], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_16_anger_57.png', 0.531816], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_10_15_anger_461.png', 0.521003], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_04_02_anger_341.png', 0.506039], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_54_58_anger_332.png', 0.501037], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_48_37_anger_198.png', 0.498919], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_43_22_anger_83.png', 0.497488], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_53_03_anger_101.png', 0.491336], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_02_35_anger_311.png', 0.487925], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_49_39_anger_220.png', 0.48707], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_57_44_anger_388.png', 0.482286], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_49_19_anger_213.png', 0.481841], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_11_43_19_anger_82.png', 0.478982], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_12_02_53_anger_489.png', 0.47724], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_08_31_anger_429.png', 0.477004], ['image_analysis/231218Exp15NewFeat/anger/2023_12_19_12_00_16_anger_438.png', 0.47635]]

In [152]:
fear_temp = [['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_44_44_fear_102.png', 0.90884], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_19_fear_320.png', 0.906334], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_46_19_fear_141.png', 0.888427]]

In [151]:
res_value_order = sorted(ALL_dic['fear'].items(), key=lambda item: item[1], reverse = True)
res_value_order = [list(x) for x in res_value_order[:30]]
print(res_value_order)

[['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_22_fear_321.png', 0.930199], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_54_17_fear_334.png', 0.92667], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_50_30_fear_242.png', 0.921012], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_41_58_fear_32.png', 0.918868], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_40_52_fear_218.png', 0.914898], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_54_23_fear_462.png', 0.911952], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_17_01_10_fear_491.png', 0.911653], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_37_02_fear_146.png', 0.910754], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_50_12_fear_235.png', 0.910513], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_44_44_fear_102.png', 0.90884], ['image_analysis/231218Exp15NewFeat/fear/2023

In [71]:
# ALL_dic
import matplotlib.pyplot as plt
NUM_OF_SELECT = 4 - 1

def get_img_array(path, show=False):
    original_image = Image.open(path)
    # use crop, by hand
    
#     print(original_image.size)
    # Define the crop area
    left = 280  # X-coordinate of the top left corner
    top = 75   # Y-coordinate of the top left corner
    right = 1000 # X-coordinate of the bottom right corner
    bottom = 700 # Y-coordinate of the bottom right corner
    crop_area = (left, top, right, bottom)

    # Crop the image
    cropped_image = original_image.crop(crop_area)
#     print(cropped_image.size)

#     tmp_img_reshape = tmp_img.resize((320, 240))
    
    if show:
        # Display the image
        plt.imshow(cropped_image)
        plt.axis('off')  # Turn off axis numbers
        plt.show()
    
    # convert the images to (R,G,B) arrays
    tmp_img_array = np.array(cropped_image)
    
    # flatten the arrays so they are 1 dimensional vectors
    tmp_img_array = tmp_img_array.flatten()
    
    # divide the arrays by 255, the maximum RGB value to make sure every value is on a 0-1 scale
    tmp_img_array = tmp_img_array/255
    
    return tmp_img_array

def cos_greedy_one_round(input_list, bound, choose, input_emotion):
    target_emotion = input_emotion
    NUM_OF_SELECT = choose
    target_emotion = input_emotion
    
    # round 1
    res_value_order = [list(x) for x in input_list[:bound]]
    
    # get all img array
    res_img_array = {}

    for i in res_value_order:
        temp_img_array = get_img_array(i[0])
        res_img_array[i[0]] = [i[1], temp_img_array]
    
    

    # compute all similarity
    data_length = len(res_img_array.keys())
    nameList = list(res_img_array.keys())
    cos_dis_result = {}
    cos_dis_result_listform = []
    counter = 0
    for i in range(data_length):
        for j in range(i + 1, data_length):
            vec1 = res_img_array[nameList[i]][1]
            vec2 = res_img_array[nameList[j]][1]
            cos_dis = cal_cosine_distance(vec1, vec2)
            if cos_dis == 0.0:
                raise Exception('ERROR, counter is ' + str(counter), i, j)
            tmp = [nameList[i], nameList[j], cos_dis]
            cos_dis_result_listform.append(tmp)
            cos_dis_result[counter] = tmp
            counter += 1
            
            
    rescale_cos_dis_result = rescale(cos_dis_result)
    
    # in this dic, I use all img name as the first id of the dic, there inner dic, which contains all the img name except itself.
    # I store the rating in the inner dic
    #     print('res_img_array', res_img_array)
    res_dic_rescaled_cos_dis = {i:{} for i in res_img_array.keys()}
    
    for i in rescale_cos_dis_result:
        tempa = i[1][0]
        tempb = i[1][1]
        temp_score = i[1][2]
        res_dic_rescaled_cos_dis[tempa][tempb] = temp_score
        res_dic_rescaled_cos_dis[tempb][tempa] = temp_score    

    # greedy search
#     print('res_dic_rescaled_cos_dis', res_dic_rescaled_cos_dis)
    
    count = 1
    finalResult = []
    finalResult.append(res_value_order[0])
#     finalResult.append(res_value_order[-1])
    x_list = []
    # print(finalResult)
    for i in range(len(finalResult)):
        x_list.append(finalResult[i][0])
    # print(x_list)
    for count in range(NUM_OF_SELECT):
        # for storing the distance from the selected list to others 
        y_list = []

        # find interest keys, which is not in x_list
        cur_keys = ''

        for i in x_list:
            if cur_keys == '':
                cur_keys = res_dic_rescaled_cos_dis[i].keys()
            else:
                type(cur_keys)
                cur_keys = cur_keys & res_dic_rescaled_cos_dis[i].keys()
        cur_dic = dict.fromkeys(cur_keys, 0)
        for k in x_list:
            assert k not in cur_dic 
        for i in cur_dic.keys():
            for j in x_list:
                cur_dic[i] += res_dic_rescaled_cos_dis[i][j]
    #     print(len(cur_keys))
        largeest_dis_target_name = max(cur_dic, key=cur_dic.get)
        pos_of_largeest_dis_target_name = [index1 for index1,value1 in 
         enumerate(res_value_order) for index2,value2 in enumerate(value1) if value2==largeest_dis_target_name][0]
        x_list.append(largeest_dis_target_name)
        finalResult.append(res_value_order[pos_of_largeest_dis_target_name])
        
    finalResult = sorted(finalResult, key=lambda x: x[1], reverse = True)
    
    return finalResult


for target_emotion in ['disgust', 'fear', 'sadness', 'surprise']:

#     # round 1
#     res_value_order = sorted(ALL_dic[target_emotion].items(), key=lambda item: item[1], reverse = True)
#     res_value_order = [list(x) for x in res_value_order[:250]]
    
#     # get all img array
#     res_img_array = {}

#     for i in res_value_order:
#         temp_img_array = get_img_array(i[0])
#         res_img_array[i[0]] = [i[1], temp_img_array]
    
    

#     # compute all similarity
#     data_length = len(res_img_array.keys())
#     nameList = list(res_img_array.keys())
#     cos_dis_result = {}
#     cos_dis_result_listform = []
#     counter = 0
#     for i in range(data_length):
#         for j in range(i + 1, data_length):
#             vec1 = res_img_array[nameList[i]][1]
#             vec2 = res_img_array[nameList[j]][1]
#             cos_dis = cal_cosine_distance(vec1, vec2)
#             if cos_dis == 0.0:
#                 raise Exception('ERROR, counter is ' + str(counter), i, j)
#             tmp = [nameList[i], nameList[j], cos_dis]
#             cos_dis_result_listform.append(tmp)
#             cos_dis_result[counter] = tmp
#             counter += 1
            
            
#     rescale_cos_dis_result = rescale(cos_dis_result)
    
#     # in this dic, I use all img name as the first id of the dic, there inner dic, which contains all the img name except itself.
#     # I store the rating in the inner dic
#     #     print('res_img_array', res_img_array)
#     res_dic_rescaled_cos_dis = {i:{} for i in res_img_array.keys()}
    
#     for i in rescale_cos_dis_result:
#         tempa = i[1][0]
#         tempb = i[1][1]
#         temp_score = i[1][2]
#         res_dic_rescaled_cos_dis[tempa][tempb] = temp_score
#         res_dic_rescaled_cos_dis[tempb][tempa] = temp_score    

#     # greedy search
# #     print('res_dic_rescaled_cos_dis', res_dic_rescaled_cos_dis)
    
#     count = 1
#     finalResult = []
#     finalResult.append(res_value_order[0])
# #     finalResult.append(res_value_order[-1])
#     x_list = []
#     # print(finalResult)
#     for i in range(len(finalResult)):
#         x_list.append(finalResult[i][0])
#     # print(x_list)
#     for count in range(NUM_OF_SELECT):
#         # for storing the distance from the selected list to others 
#         y_list = []

#         # find interest keys, which is not in x_list
#         cur_keys = ''

#         for i in x_list:
#             if cur_keys == '':
#                 cur_keys = res_dic_rescaled_cos_dis[i].keys()
#             else:
#                 type(cur_keys)
#                 cur_keys = cur_keys & res_dic_rescaled_cos_dis[i].keys()
#         cur_dic = dict.fromkeys(cur_keys, 0)
#         for k in x_list:
#             assert k not in cur_dic 
#         for i in cur_dic.keys():
#             for j in x_list:
#                 cur_dic[i] += res_dic_rescaled_cos_dis[i][j]
#     #     print(len(cur_keys))
#         largeest_dis_target_name = max(cur_dic, key=cur_dic.get)
#         pos_of_largeest_dis_target_name = [index1 for index1,value1 in 
#          enumerate(res_value_order) for index2,value2 in enumerate(value1) if value2==largeest_dis_target_name][0]
#         x_list.append(largeest_dis_target_name)
#         finalResult.append(res_value_order[pos_of_largeest_dis_target_name])
        
#     finalResult = sorted(finalResult, key=lambda x: x[1], reverse = True)
    
    # round 1
    res_value_order = sorted(ALL_dic[target_emotion].items(), key=lambda item: item[1], reverse = True)
    res_value_order = [list(x) for x in res_value_order[:250]]
    r1 = cos_greedy_one_round(res_value_order, bound=10, choose=4-1, input_emotion=target_emotion)
    
#     # round 2
#     r2 = cos_greedy_one_round(r1, bound=30, choose=10, input_emotion=target_emotion)
    
#     # round 3
#     r3 = cos_greedy_one_round(r2, bound=-1, choose=4, input_emotion=target_emotion)
    
    
    print(r1)

max is: 0.0018795780651115201 
min is: 0.0003538606524303711
rescaling...
rescaled.
[['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_38_10_disgust_446.png', 0.935409], ['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_12_37_disgust_192.png', 0.933728], ['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_13_54_disgust_215.png', 0.922659], ['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_32_44_disgust_325.png', 0.922036]]
max is: 0.0014828953341958773 
min is: 0.00037171640337874123
rescaling...
rescaled.
[['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_22_fear_321.png', 0.930199], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_40_52_fear_218.png', 0.914898], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_17_01_10_fear_491.png', 0.911653]]
max is: 0.0008678392412105884 
min is: 0.000331191713080603
rescaling...
rescaled.
[['image_analysis/231219Ex

In [153]:
import shutil
def copyImageToNewFolder(imglist, exp):
    path = f"image_analysis/top50{exp}/"
    if not os.path.exists(path):
        # Create a new directory because it does not exist
        os.makedirs(path)
    for i, image in enumerate(imglist): 
        tmp = image.split('/')[-1]
        tmpcsv = image.split('/')[-1][:-4] + '_axes_data.csv'
        origin_path = os.path.join(*image.split('/')[:-1])
#         print(origin_path)
        shutil.copy(image, os.path.join(path, f'{i+1}_{tmp}'))
        shutil.copy(os.path.join(origin_path, tmpcsv), os.path.join(path, f'{i+1}_{tmpcsv}'))
    print('Done.')

    
res_value_order = [['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_14_42_happiness_337.png', 0.956833], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_32_03_happiness_462.png', 0.935815], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_22_25_happiness_469.png', 0.916838], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_08_07_happiness_218.png', 0.914777], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_23_58_happiness_275.png', 0.910313], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_17_01_happiness_378.png', 0.902309], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_11_01_happiness_270.png', 0.894584], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_10_21_happiness_258.png', 0.892963], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_25_12_happiness_305.png', 0.888788], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_19_50_happiness_427.png', 0.888061], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_15_29_happiness_351.png', 0.887657], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_16_21_happiness_367.png', 0.884125], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_01_20_happiness_91.png', 0.883208], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_12_49_happiness_304.png', 0.881794], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_26_26_happiness_334.png', 0.88104], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_24_28_happiness_287.png', 0.875617], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_25_49_happiness_319.png', 0.87008], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_18_47_happiness_409.png', 0.866897], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_25_01_happiness_301.png', 0.864147], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_24_06_happiness_278.png', 0.860579], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_05_12_happiness_165.png', 0.860237], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_26_24_happiness_333.png', 0.858731], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_20_13_happiness_433.png', 0.85833], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_29_21_happiness_402.png', 0.854663], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_00_00_happiness_66.png', 0.851545], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_10_51_happiness_267.png', 0.851301], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_23_37_happiness_489.png', 0.850103], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_07_50_happiness_213.png', 0.849319], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_17_32_happiness_121.png', 0.838519], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_17_34_happiness_388.png', 0.8365]]
# res_value_order = fear_temp

res_value_order = [x[0] for x in res_value_order]
copyImageToNewFolder(res_value_order, 'happiness')


Done.


In [110]:
# target_emotion = 'happiness'
# res_value_order = sorted(ALL_dic[target_emotion].items(), key=lambda item: item[1], reverse = True)
# res_value_order = [list(x) for x in res_value_order[:250]]
# print(res_value_order)


In [117]:
mypath = 'image_analysis/top50happiness/'
# mypath_2 = 'image_analysis/231218Exp15NewFeat/happiness/'
# newhappy = [i for i in os.listdir(mypath) if 'png' in i]
newhappy = [['11_2023_12_19_13_15_29_happiness_351.png',0.887657], ['23_2023_12_19_13_20_13_happiness_433.png', 0.85833], ['4_2023_12_19_13_08_07_happiness_218.png', 0.914777],['30_2023_12_19_13_17_34_happiness_388.png', 0.8365],['12_2023_12_19_13_16_21_happiness_367.png', 0.884125],['15_2023_12_19_15_26_26_happiness_334.png', 0.4834],['10_2023_12_19_13_19_50_happiness_427.png', 0.888061], ['1_2023_12_19_13_14_42_happiness_337.png', 0.956833],['26_2023_12_19_13_10_51_happiness_267.png', 0.851301],['14_2023_12_19_13_12_49_happiness_304.png', 0.881794],['20_2023_12_19_15_24_06_happiness_278.png', 0.910493]]
for i, _ in enumerate(newhappy):
    newhappy[i][0] = os.path.join(mypath, newhappy[i][0])
    
newhappy = sorted(newhappy, key=lambda x:x[1], reverse=True)
print(newhappy)


[['image_analysis/top50happiness/1_2023_12_19_13_14_42_happiness_337.png', 0.956833], ['image_analysis/top50happiness/4_2023_12_19_13_08_07_happiness_218.png', 0.914777], ['image_analysis/top50happiness/20_2023_12_19_15_24_06_happiness_278.png', 0.910493], ['image_analysis/top50happiness/10_2023_12_19_13_19_50_happiness_427.png', 0.888061], ['image_analysis/top50happiness/11_2023_12_19_13_15_29_happiness_351.png', 0.887657], ['image_analysis/top50happiness/12_2023_12_19_13_16_21_happiness_367.png', 0.884125], ['image_analysis/top50happiness/14_2023_12_19_13_12_49_happiness_304.png', 0.881794], ['image_analysis/top50happiness/23_2023_12_19_13_20_13_happiness_433.png', 0.85833], ['image_analysis/top50happiness/26_2023_12_19_13_10_51_happiness_267.png', 0.851301], ['image_analysis/top50happiness/30_2023_12_19_13_17_34_happiness_388.png', 0.8365], ['image_analysis/top50happiness/15_2023_12_19_15_26_26_happiness_334.png', 0.4834]]


In [118]:
# ALL_dic
# happiness
import matplotlib.pyplot as plt
NUM_OF_SELECT = 4 - 1

def get_img_array(path, show=False):
    original_image = Image.open(path)
    # use crop, by hand
    
#     print(original_image.size)
    # Define the crop area
    left = 280  # X-coordinate of the top left corner
    top = 75   # Y-coordinate of the top left corner
    right = 1000 # X-coordinate of the bottom right corner
    bottom = 700 # Y-coordinate of the bottom right corner
    crop_area = (left, top, right, bottom)

    # Crop the image
    cropped_image = original_image.crop(crop_area)
#     print(cropped_image.size)

#     tmp_img_reshape = tmp_img.resize((320, 240))
    
    if show:
        # Display the image
        plt.imshow(cropped_image)
        plt.axis('off')  # Turn off axis numbers
        plt.show()
    
    # convert the images to (R,G,B) arrays
    tmp_img_array = np.array(cropped_image)
    
    # flatten the arrays so they are 1 dimensional vectors
    tmp_img_array = tmp_img_array.flatten()
    
    # divide the arrays by 255, the maximum RGB value to make sure every value is on a 0-1 scale
    tmp_img_array = tmp_img_array/255
    
    return tmp_img_array

# for target_emotion in ['disgust', 'fear', 'sadness', 'surprise']:
# for target_emotion in ['anger']:
for target_emotion in ['happiness']:
    res_value_order = sorted(ALL_dic[target_emotion].items(), key=lambda item: item[1], reverse = True)
    res_value_order = [list(x) for x in res_value_order[:150]]
    res_value_order = newhappy

    
#     print(target_emotion)
#     print(res_value_order)
#     print()
    
    # get all img array
    res_img_array = {}

    for i in res_value_order:
        temp_img_array = get_img_array(i[0])
        res_img_array[i[0]] = [i[1], temp_img_array]
    
    

    # compute all similarity
    data_length = len(res_img_array.keys())
    nameList = list(res_img_array.keys())
    cos_dis_result = {}
    cos_dis_result_listform = []
    counter = 0
    for i in range(data_length):
        for j in range(i + 1, data_length):
            vec1 = res_img_array[nameList[i]][1]
            vec2 = res_img_array[nameList[j]][1]
            cos_dis = cal_cosine_distance(vec1, vec2)
            if cos_dis == 0.0:
                raise Exception('ERROR, counter is ' + str(counter), i, j)
            tmp = [nameList[i], nameList[j], cos_dis]
            cos_dis_result_listform.append(tmp)
            cos_dis_result[counter] = tmp
            counter += 1
            
            
    rescale_cos_dis_result = rescale(cos_dis_result)
    
    # in this dic, I use all img name as the first id of the dic, there inner dic, which contains all the img name except itself.
    # I store the rating in the inner dic
    #     print('res_img_array', res_img_array)
    res_dic_rescaled_cos_dis = {i:{} for i in res_img_array.keys()}
    
    for i in rescale_cos_dis_result:
        tempa = i[1][0]
        tempb = i[1][1]
        temp_score = i[1][2]
        res_dic_rescaled_cos_dis[tempa][tempb] = temp_score
        res_dic_rescaled_cos_dis[tempb][tempa] = temp_score    

    # greedy search
#     print('res_dic_rescaled_cos_dis', res_dic_rescaled_cos_dis)
    
    count = 1
    finalResult = []
    finalResult.append(res_value_order[0])
#     finalResult.append(res_value_order[-1])
    x_list = []
    # print(finalResult)
    for i in range(len(finalResult)):
        x_list.append(finalResult[i][0])
    # print(x_list)
    for count in range(NUM_OF_SELECT):
        # for storing the distance from the selected list to others 
        y_list = []

        # find interest keys, which is not in x_list
        cur_keys = ''

        for i in x_list:
            if cur_keys == '':
                cur_keys = res_dic_rescaled_cos_dis[i].keys()
            else:
                type(cur_keys)
                cur_keys = cur_keys & res_dic_rescaled_cos_dis[i].keys()
        cur_dic = dict.fromkeys(cur_keys, 0)
        for k in x_list:
            assert k not in cur_dic 
        for i in cur_dic.keys():
            for j in x_list:
                cur_dic[i] += res_dic_rescaled_cos_dis[i][j]
    #     print(len(cur_keys))
        largeest_dis_target_name = max(cur_dic, key=cur_dic.get)
        pos_of_largeest_dis_target_name = [index1 for index1,value1 in 
         enumerate(res_value_order) for index2,value2 in enumerate(value1) if value2==largeest_dis_target_name][0]
        x_list.append(largeest_dis_target_name)
        finalResult.append(res_value_order[pos_of_largeest_dis_target_name])
        
    finalResult = sorted(finalResult, key=lambda x: x[1], reverse = True)
    print(finalResult)

max is: 0.0015063026578550076 
min is: 0.0004030153068240727
rescaling...
rescaled.
[['image_analysis/top50happiness/1_2023_12_19_13_14_42_happiness_337.png', 0.956833], ['image_analysis/top50happiness/20_2023_12_19_15_24_06_happiness_278.png', 0.910493], ['image_analysis/top50happiness/30_2023_12_19_13_17_34_happiness_388.png', 0.8365], ['image_analysis/top50happiness/15_2023_12_19_15_26_26_happiness_334.png', 0.4834]]


In [ ]:
[['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_14_42_happiness_337.png', 0.956833], 
 ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_15_24_06_happiness_278.png', 0.910493], 
 ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_17_34_happiness_388.png', 0.8365], 
 ['image_analysis/2023_12_19_15_26_26_happiness_334.png', 0.4834]]

# For Copy files

## selected for exp 2

In [155]:
# selected for exp 2
selected_Online_Exp_2 = {}
selected_Online_Exp_2['anger'] = [['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_58_49_anger_230.png', 0.877257], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_19_anger_58.png', 0.8548], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_10_44_anger_469.png', 0.825715], ['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_46_anger_69.png', 0.862081]]
selected_Online_Exp_2['disgust'] = [['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_38_10_disgust_446.png', 0.935409], ['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_12_37_disgust_192.png', 0.933728], ['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_13_54_disgust_215.png', 0.922659], ['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_32_44_disgust_325.png', 0.922036]]
# selected_Online_Exp_2['fear'] = [['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_22_fear_321.png', 0.930199], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_40_52_fear_218.png', 0.914898], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_17_01_10_fear_491.png', 0.911653]]
selected_Online_Exp_2['fear'] = [['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_44_44_fear_102.png', 0.90884], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_19_fear_320.png', 0.906334], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_46_19_fear_141.png', 0.888427]]
selected_Online_Exp_2['happiness'] = [['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_14_42_happiness_337.png', 0.956833], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_24_06_happiness_278.png', 0.910493], ['image_analysis/231218Exp15NewFeat/happiness/2023_12_19_13_17_34_happiness_388.png', 0.8365], ['image_analysis/231219Exp16Feat037/happiness/2023_12_19_15_26_26_happiness_334.png', 0.4834]]
selected_Online_Exp_2['sadness'] = [['image_analysis/231219Exp16Feat037/sadness/2023_12_19_15_51_40_sadness_370.png', 0.92779], ['image_analysis/231218Exp15NewFeat/sadness/2023_12_19_13_28_55_sadness_79.png', 0.904773], ['image_analysis/231219Exp16Feat037/sadness/2023_12_19_15_55_02_sadness_437.png', 0.904087], ['image_analysis/231219Exp16Feat037/sadness/2023_12_19_15_47_59_sadness_293.png', 0.901349]]
selected_Online_Exp_2['surprise'] = [['image_analysis/231218Exp15NewFeat/surprise/2023_12_19_13_57_55_surprise_101.png', 0.998629], ['image_analysis/231219Exp16Feat037/surprise/2023_12_19_16_00_58_surprise_64.png', 0.997973], ['image_analysis/231219Exp16Feat037/surprise/2023_12_19_16_19_25_surprise_496.png', 0.997658], ['image_analysis/231218Exp15NewFeat/surprise/2023_12_19_13_55_34_surprise_55.png', 0.997651]]



In [156]:
for k, v in selected_Online_Exp_2.items():
    print(k)
    for i in v:
        print(i)
    print()

anger
['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_58_49_anger_230.png', 0.877257]
['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_19_anger_58.png', 0.8548]
['image_analysis/231219Exp16Feat037/anger/2023_12_19_15_10_44_anger_469.png', 0.825715]
['image_analysis/231219Exp16Feat037/anger/2023_12_19_14_51_46_anger_69.png', 0.862081]

disgust
['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_38_10_disgust_446.png', 0.935409]
['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_12_37_disgust_192.png', 0.933728]
['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_13_54_disgust_215.png', 0.922659]
['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_32_44_disgust_325.png', 0.922036]

fear
['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642]
['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_44_44_fear_102.png', 0.90884]
['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_19_fear_320.png', 0.906334]
['imag

In [157]:
COPY_FLAG = 1
NUM_OF_SELECT = 4

folderName = 'image_analysis/231225Exp17NewFeat'
print(folderName)
sum_folderName = os.path.join(folderName, 'Online_exp2_selected_{}'.format(NUM_OF_SELECT))
print(sum_folderName)
img_only_folderName = os.path.join(folderName, 'Exp2_selected_img_only_{}'.format(NUM_OF_SELECT))
print(img_only_folderName)
if COPY_FLAG and not os.path.exists(sum_folderName):
    try:
        os.mkdir(sum_folderName)
    except Exception as e:
        print(e)
if COPY_FLAG and not os.path.exists(img_only_folderName):
    try:
        os.mkdir(img_only_folderName)
    except Exception as e:
        print(e)

import shutil

image_analysis/231225Exp17NewFeat
image_analysis/231225Exp17NewFeat/Online_exp2_selected_4
image_analysis/231225Exp17NewFeat/Exp2_selected_img_only_4


In [146]:
# ALL_dic
import matplotlib.pyplot as plt
NUM_OF_SELECT = 4 - 1

def get_img_array(path, show=False):
    original_image = Image.open(path)
    # use crop, by hand
    
#     print(original_image.size)
    # Define the crop area
    left = 280  # X-coordinate of the top left corner
    top = 75   # Y-coordinate of the top left corner
    right = 1000 # X-coordinate of the bottom right corner
    bottom = 700 # Y-coordinate of the bottom right corner
    crop_area = (left, top, right, bottom)

    # Crop the image
    cropped_image = original_image.crop(crop_area)
#     print(cropped_image.size)

#     tmp_img_reshape = tmp_img.resize((320, 240))
    
    if show:
        # Display the image
        plt.imshow(cropped_image)
        plt.axis('off')  # Turn off axis numbers
        plt.show()
    
    # convert the images to (R,G,B) arrays
    tmp_img_array = np.array(cropped_image)
    
    # flatten the arrays so they are 1 dimensional vectors
    tmp_img_array = tmp_img_array.flatten()
    
    # divide the arrays by 255, the maximum RGB value to make sure every value is on a 0-1 scale
    tmp_img_array = tmp_img_array/255
    
    return tmp_img_array

def cos_greedy_one_round(input_list, bound, choose, input_emotion):
    target_emotion = input_emotion
    NUM_OF_SELECT = choose
    target_emotion = input_emotion
    
    # round 1
    res_value_order = [list(x) for x in input_list[:bound]]
    
    # get all img array
    res_img_array = {}

    for i in res_value_order:
        temp_img_array = get_img_array(i[0])
        res_img_array[i[0]] = [i[1], temp_img_array]
    
    

    # compute all similarity
    data_length = len(res_img_array.keys())
    nameList = list(res_img_array.keys())
    cos_dis_result = {}
    cos_dis_result_listform = []
    counter = 0
    for i in range(data_length):
        for j in range(i + 1, data_length):
            vec1 = res_img_array[nameList[i]][1]
            vec2 = res_img_array[nameList[j]][1]
            cos_dis = cal_cosine_distance(vec1, vec2)
            if cos_dis == 0.0:
                raise Exception('ERROR, counter is ' + str(counter), i, j)
            tmp = [nameList[i], nameList[j], cos_dis]
            cos_dis_result_listform.append(tmp)
            cos_dis_result[counter] = tmp
            counter += 1
            
            
    rescale_cos_dis_result = rescale(cos_dis_result)
    
    # in this dic, I use all img name as the first id of the dic, there inner dic, which contains all the img name except itself.
    # I store the rating in the inner dic
    #     print('res_img_array', res_img_array)
    res_dic_rescaled_cos_dis = {i:{} for i in res_img_array.keys()}
    
    for i in rescale_cos_dis_result:
        tempa = i[1][0]
        tempb = i[1][1]
        temp_score = i[1][2]
        res_dic_rescaled_cos_dis[tempa][tempb] = temp_score
        res_dic_rescaled_cos_dis[tempb][tempa] = temp_score    

    # greedy search
#     print('res_dic_rescaled_cos_dis', res_dic_rescaled_cos_dis)
    
    count = 1
    finalResult = []
    finalResult.append(res_value_order[0])
#     finalResult.append(res_value_order[-1])
    x_list = []
    # print(finalResult)
    for i in range(len(finalResult)):
        x_list.append(finalResult[i][0])
    # print(x_list)
    for count in range(NUM_OF_SELECT):
        # for storing the distance from the selected list to others 
        y_list = []

        # find interest keys, which is not in x_list
        cur_keys = ''

        for i in x_list:
            if cur_keys == '':
                cur_keys = res_dic_rescaled_cos_dis[i].keys()
            else:
                type(cur_keys)
                cur_keys = cur_keys & res_dic_rescaled_cos_dis[i].keys()
        cur_dic = dict.fromkeys(cur_keys, 0)
        for k in x_list:
            assert k not in cur_dic 
        for i in cur_dic.keys():
            for j in x_list:
                cur_dic[i] += res_dic_rescaled_cos_dis[i][j]
    #     print(len(cur_keys))
        largeest_dis_target_name = max(cur_dic, key=cur_dic.get)
        pos_of_largeest_dis_target_name = [index1 for index1,value1 in 
         enumerate(res_value_order) for index2,value2 in enumerate(value1) if value2==largeest_dis_target_name][0]
        x_list.append(largeest_dis_target_name)
        finalResult.append(res_value_order[pos_of_largeest_dis_target_name])
        
    finalResult = sorted(finalResult, key=lambda x: x[1], reverse = True)
    
    return finalResult


for target_emotion in ['disgust', 'fear', 'sadness', 'surprise']:

    
    # round 1
    res_value_order = sorted(ALL_dic[target_emotion].items(), key=lambda item: item[1], reverse = True)
    res_value_order = [list(x) for x in res_value_order[:250]]
    r1 = cos_greedy_one_round(res_value_order, bound=10, choose=4-1, input_emotion=target_emotion)
    
#     # round 2
#     r2 = cos_greedy_one_round(r1, bound=30, choose=10, input_emotion=target_emotion)
    
#     # round 3
#     r3 = cos_greedy_one_round(r2, bound=-1, choose=4, input_emotion=target_emotion)
    
    
    print(r1)

max is: 0.0018795780651115201 
min is: 0.0003538606524303711
rescaling...
rescaled.
[['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_38_10_disgust_446.png', 0.935409], ['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_12_37_disgust_192.png', 0.933728], ['image_analysis/231218Exp15NewFeat/disgust/2023_12_19_12_13_54_disgust_215.png', 0.922659], ['image_analysis/231219Exp16Feat037/disgust/2023_12_19_16_32_44_disgust_325.png', 0.922036]]
max is: 0.0014828953341958773 
min is: 0.00037171640337874123
rescaling...
rescaled.
[['image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png', 0.938642], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_22_fear_321.png', 0.930199], ['image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_40_52_fear_218.png', 0.914898], ['image_analysis/231219Exp16Feat037/fear/2023_12_19_17_01_10_fear_491.png', 0.911653]]
max is: 0.0008678392412105884 
min is: 0.000331191713080603
rescaling...
rescaled.
[['image_analysis/231219Ex

In [158]:
# for target_emotion in  ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']:
for target_emotion in  ['fear']:
    for i in selected_Online_Exp_2[target_emotion]:
        # i is fig name
        print(i[0], i[1])
        img_file_name = i[0]
        csv_file_name = i[0][:-4] + '.csv'
        axes_file_name = i[0][:-4] + '_axes_data.csv'
        emotion_file_name = i[0][:-4] + '_emotion.csv'
        
        cur_sum_folderName = os.path.join(sum_folderName, target_emotion)
        if not os.path.exists(cur_sum_folderName):
            os.mkdir(cur_sum_folderName)

        # all files
        if COPY_FLAG and not os.path.exists(os.path.join(cur_sum_folderName, i[0])):
            shutil.copy(img_file_name, cur_sum_folderName)
            print('copied0')
        if COPY_FLAG and not os.path.exists(os.path.join(cur_sum_folderName, i[0][:-4] + '.csv')):
            shutil.copy(csv_file_name, cur_sum_folderName)
            print('copied1')
        if COPY_FLAG and not os.path.exists(os.path.join(cur_sum_folderName, i[0][:-4] + '_axes_data.csv')):
            shutil.copy(axes_file_name, cur_sum_folderName)
            print('copied2')
        if COPY_FLAG and not os.path.exists(os.path.join(cur_sum_folderName, i[0][:-4] + '_emotion.csv')):
            shutil.copy(emotion_file_name, cur_sum_folderName)
            print('copied3')

        if COPY_FLAG and not os.path.exists(os.path.join(img_only_folderName, i[0])):
            shutil.copy(img_file_name, img_only_folderName)
            print('copied4')

image_analysis/231219Exp16Feat037/fear/2023_12_19_16_55_14_fear_356.png 0.938642
copied0
copied1
copied2
copied3
copied4
image_analysis/231219Exp16Feat037/fear/2023_12_19_16_44_44_fear_102.png 0.90884
copied0
copied1
copied2
copied3
copied4
image_analysis/231218Exp15NewFeat/fear/2023_12_19_12_46_19_fear_320.png 0.906334
copied0
copied1
copied2
copied3
copied4
image_analysis/231219Exp16Feat037/fear/2023_12_19_16_46_19_fear_141.png 0.888427
copied0
copied1
copied2
copied3
copied4


In [51]:
COPY_FLAG = 0 # 1 is copy, 0 - no copy
NUM_OF_SELECT = 5 - 1

# get all img array, for anger
for target_emotion in ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']:
    folderName = os.path.join('image_analysis/211223ExpHumanoidRaw/Exp7HumanoidRaw/', target_emotion)
    ls_name = os.listdir(folderName)
#     print(ls_name)
    result_dic = {}

    for i in ls_name:
        if 'emotion' in i:
            mypath = os.path.join(folderName, i)
            temp = pd.read_csv(mypath)
            result = temp[target_emotion][0]
            result = round(result, 6)
            fileName = temp['input'][0]
            result_dic[i[:-12] + '.png'] = result
    # sort in value order 
    res_value_order = sorted(result_dic.items(), key=lambda item: item[1], reverse = True)
    
    ##### choose top 10
    res_value_order = res_value_order[:10]
    print(target_emotion)
#     print(len(res_value_order))
#     print(len(res_value_order))
    for i in res_value_order:
        print(i)
    print()

anger
('2021_12_23_15_32_12_anger_772.png', 0.936236)
('2021_12_23_15_28_50_anger_402.png', 0.933472)
('2021_12_23_15_28_56_anger_412.png', 0.932501)
('2021_12_23_15_27_09_anger_212.png', 0.931665)
('2021_12_23_15_30_51_anger_622.png', 0.929667)
('2021_12_23_15_31_17_anger_672.png', 0.927082)
('2021_12_23_15_32_40_anger_822.png', 0.922744)
('2021_12_23_15_29_01_anger_422.png', 0.918208)
('2021_12_23_15_30_17_anger_562.png', 0.914131)
('2021_12_23_15_33_23_anger_902.png', 0.913679)

disgust
('2021_12_23_15_42_21_disgust_822.png', 0.827066)
('2021_12_23_15_42_37_disgust_852.png', 0.817449)
('2021_12_23_15_36_37_disgust_182.png', 0.804087)
('2021_12_23_15_40_02_disgust_562.png', 0.803605)
('2021_12_23_15_38_53_disgust_432.png', 0.797357)
('2021_12_23_15_42_00_disgust_782.png', 0.789126)
('2021_12_23_15_38_14_disgust_362.png', 0.788378)
('2021_12_23_15_42_43_disgust_862.png', 0.767849)
('2021_12_23_15_36_58_disgust_222.png', 0.76772)
('2021_12_23_15_36_16_disgust_142.png', 0.756644)

fear


In [48]:
COPY_FLAG = 0 # 1 is copy, 0 - no copy
NUM_OF_SELECT = 5 - 1

# get all img array, for anger
for target_emotion in ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']:
    folderName = os.path.join('image_analysis/211223ExpHumanoidRaw/Exp7HumanoidRaw/', target_emotion)
    ls_name = os.listdir(folderName)
#     print(ls_name)
    result_dic = {}

    for i in ls_name:
        if 'emotion' in i:
            mypath = os.path.join(folderName, i)
            temp = pd.read_csv(mypath)
            result = temp[target_emotion][0]
            result = round(result, 6)
            fileName = temp['input'][0]
            result_dic[i[:-12] + '.png'] = result
    # sort in value order 
    res_value_order = sorted(result_dic.items(), key=lambda item: item[1], reverse = True)
    
    ##### choose top 10
    res_value_order = res_value_order[:10]
    print(len(res_value_order))
#     print(len(res_value_order))
#     print(res_value_order)
#     break
    

    
    # get all img array
    res_img_array = {}

    for i in res_value_order:
        temp_img_array = get_img_array(os.path.join(folderName, i[0]))
        res_img_array[i[0]] = [i[1], temp_img_array]

    # compute all similarity
    data_length = len(res_img_array.keys())
    nameList = list(res_img_array.keys())
    cos_dis_result = {}
    cos_dis_result_listform = []
    counter = 0
    for i in range(data_length):
        for j in range(i + 1, data_length):
            vec1 = res_img_array[nameList[i]][1]
            vec2 = res_img_array[nameList[j]][1]
            cos_dis = cal_cosine_distance(vec1, vec2)
            if cos_dis == 0.0:
                raise Exception('ERROR, counter is ' + str(counter), i, j)
            tmp = [nameList[i], nameList[j], cos_dis]
            cos_dis_result_listform.append(tmp)
            cos_dis_result[counter] = tmp
            counter += 1
    rescale_cos_dis_result = rescale(cos_dis_result)

    # in this dic, I use all img name as the first id of the dic, there inner dic, which contains all the img name except itself.
    # I store the rating in the inner dic
    #     print('res_img_array', res_img_array)
    res_dic_rescaled_cos_dis = {i:{} for i in res_img_array.keys()}
    
    for i in rescale_cos_dis_result:
        tempa = i[1][0]
        tempb = i[1][1]
        temp_score = i[1][2]
        res_dic_rescaled_cos_dis[tempa][tempb] = temp_score
        res_dic_rescaled_cos_dis[tempb][tempa] = temp_score    

    # greedy search
#     print('res_dic_rescaled_cos_dis', res_dic_rescaled_cos_dis)
    
    count = 1
    finalResult = []
    finalResult.append(res_value_order[0])
#     finalResult.append(res_value_order[-1])
    x_list = []
    # print(finalResult)
    for i in range(len(finalResult)):
        x_list.append(finalResult[i][0])
    # print(x_list)
    for count in range(NUM_OF_SELECT):
        # for storing the distance from the selected list to others 
        y_list = []

        # find interest keys, which is not in x_list
        cur_keys = ''

        for i in x_list:
            if cur_keys == '':
                cur_keys = res_dic_rescaled_cos_dis[i].keys()
            else:
                type(cur_keys)
                cur_keys = cur_keys & res_dic_rescaled_cos_dis[i].keys()
        cur_dic = dict.fromkeys(cur_keys, 0)
        for k in x_list:
            assert k not in cur_dic 
        for i in cur_dic.keys():
            for j in x_list:
                cur_dic[i] += res_dic_rescaled_cos_dis[i][j]
    #     print(len(cur_keys))
        largeest_dis_target_name = max(cur_dic, key=cur_dic.get)
        pos_of_largeest_dis_target_name = [index1 for index1,value1 in 
         enumerate(res_value_order) for index2,value2 in enumerate(value1) if value2==largeest_dis_target_name][0]
        x_list.append(largeest_dis_target_name)
        finalResult.append(res_value_order[pos_of_largeest_dis_target_name])


    # Copy files
    sum_folderName = os.path.join(folderName, 'selected{}'.format(NUM_OF_SELECT))
    img_only_folderName = os.path.join(folderName, 'selected_imgonly{}'.format(NUM_OF_SELECT))
    if COPY_FLAG and not os.path.exists(sum_folderName):
        try:
            os.mkdir(sum_folderName)
        except Exception as e:
            print(e)
    if COPY_FLAG and not os.path.exists(img_only_folderName):
        try:
            os.mkdir(img_only_folderName)
        except Exception as e:
            print(e)

    import shutil
    for i in finalResult:
        # i is fig name
        print(i[0], i[1])
        img_file_name = os.path.join(folderName, i[0])
        csv_file_name = os.path.join(folderName, i[0][:-4] + '.csv')
        axes_file_name = os.path.join(folderName, i[0][:-4] + '_axes_data.csv')
        emotion_file_name = os.path.join(folderName, i[0][:-4] + '_emotion.csv')

        # all files
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0])):
            shutil.copy(img_file_name, sum_folderName)
            print('copied0')
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0][:-4] + '.csv')):
            shutil.copy(csv_file_name, sum_folderName)
            print('copied1')
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0][:-4] + '_axes_data.csv')):
            shutil.copy(axes_file_name, sum_folderName)
            print('copied2')
        if COPY_FLAG and not os.path.exists(os.path.join(sum_folderName, i[0][:-4] + '_emotion.csv')):
            shutil.copy(emotion_file_name, sum_folderName)
            print('copied3')

        if COPY_FLAG and not os.path.exists(os.path.join(img_only_folderName, i[0])):
            shutil.copy(img_file_name, img_only_folderName)
            print('copied4')

10
max is: 0.0013050432051180394 
min is: 9.090029064962035e-05
rescaling...
rescaled.
2021_12_23_15_32_12_anger_772.png 0.936236
2021_12_23_15_33_23_anger_902.png 0.913679
2021_12_23_15_30_17_anger_562.png 0.914131
2021_12_23_15_27_09_anger_212.png 0.931665
2021_12_23_15_30_51_anger_622.png 0.929667
10
max is: 0.0011884868817438887 
min is: 0.00012923191170677306
rescaling...
rescaled.
2021_12_23_15_42_21_disgust_822.png 0.827066
2021_12_23_15_36_16_disgust_142.png 0.756644
2021_12_23_15_42_43_disgust_862.png 0.767849
2021_12_23_15_40_02_disgust_562.png 0.803605
2021_12_23_15_42_00_disgust_782.png 0.789126
10
max is: 0.0008899584555609463 
min is: 6.940063140969954e-05
rescaling...
rescaled.
2021_12_23_15_50_44_fear_792.png 0.836327
2021_12_23_15_50_03_fear_712.png 0.741456
2021_12_23_15_51_09_fear_842.png 0.619278
2021_12_23_15_52_23_fear_982.png 0.695252
2021_12_23_15_49_53_fear_692.png 0.651133
10
max is: 0.0014465283592445255 
min is: 0.00012206624062360838
rescaling...
rescaled.


'2023_02_10_15_12_04_anger_8.png'

In [86]:
print(rescale_cos_dis_result[0])
print(rescale_cos_dis_result[-1])

(55302, ['2023_02_10_15_30_46_anger_490.png', '2023_02_10_15_12_04_anger_8.png', 1.0])
(1509, ['2023_02_10_15_24_05_anger_335.png', '2023_02_10_15_24_08_anger_336.png', 0.0])


[[0, ('2023_02_10_15_28_04_anger_433.png', 0.890863)],
 [0, ('2023_02_10_15_25_54_anger_381.png', 0.000222)]]

In [139]:
cos_dis_result_listform[0]
cos_dis_result_listform[1]

[('2023_02_10_15_28_04_anger_433.png', 0.890863),
 ('2023_02_10_15_28_40_anger_446.png', 0.879018),
 6.64947043207409e-05]

In [137]:
# construct a 500 x 500 matrix
mat_2d = []
print(len(cos_dis_result_listform))
for i in range(500*500):
    mat_2d.append(cos_dis_result_listform[i][2])
ary = np.array(mat_2d).reshape(500, 500)

124750


IndexError: list index out of range

## For fast test

In [28]:
target_emotion = 'happiness'
print('Final selection for ' + target_emotion, finalResult)

# show the result
finalResult = sorted(finalResult, key=lambda x:x[0])
print('Final selection for ' + target_emotion, finalResult)
# Copy files
sum_folderName = os.path.join("image_analysis/211223ExpHumanoidRaw/Exp7HumanoidRaw/", target_emotion, '/')
if not os.path.exists(sum_folderName):
    try:
        os.mkdir(sum_folderName)
    except Exception as e:
        print(e)
import shutil
for i in finalResult:
    print(folderName)
    temp = pd.read_csv(folderName + i[1][:-4] + '_emotion.csv')
    targetID = RCSystem.get_target(target_emotion)
    temp_result = temp.iloc[:, -8:].iloc[0, targetID]
    print(i[0], i[1][:-4]+'.png', temp_result)
    tempFileAU = i[1]
    tempFileFig = i[1][:-4]+'.png'
    tempFilePyfeat = i[1][:-4] + '_emotion.csv'
#         shutil.copy(folderNameAU + tempFileAU, sum_folderName)
#         shutil.copy(folderName + tempFileFig, sum_folderName)
#         shutil.copy(folderName + tempFilePyfeat, sum_folderName)

Final selection for happiness [[0, ('2021_12_23_15_32_12_anger_772.png', 0.936236)]]
Final selection for happiness [[0, ('2021_12_23_15_32_12_anger_772.png', 0.936236)]]


TypeError: can only concatenate str (not "tuple") to str

max is: 0.003410925908368223 
min is: 6.420692183173315e-05
rescaling...
rescaled.


In [58]:
# for i in range(5):
#     print(test_rescle[i])

# for i in range(-5, 0):
#     print(test_rescle[i])

In [24]:
# greedy search

count = 1
#     print('top_name_list', top_name_list, '\n')
finalResult = []
finalResult.append([0, res_value_order[0]])

# construct a 10 x 10 matrix
mat_2d = []
for i in range(100):
    mat_2d.append(result_AU2[i][2])
ary = np.array(mat_2d).reshape(10, 10)

#     finalResult.append([2, top_name_list[2]])
while count < 5:
    x_list = []
    y_list = []
#         print()
    for i in range(len(finalResult)):
        x_list.append(finalResult[i][0])

    for i in x_list:
        y_list.append(ary[i])
#         print(y_list)
#     print(ary[i])
#     print(np.argmax(ary[i]))
    sum_v = np.zeros(10)
    for i in y_list:
        sum_v += i
    for i in x_list:
        sum_v[i] = 0
    y = np.argmax(sum_v)
    finalResult.append([y, top_name_list[y]])
    count += 1
    print('x_list', x_list)
#         print(sum_v, y)
#         np.partition(k.flatten(), -2)[-2]
#         print(ary[x].index(max(ary[x])))
#     print(max(ary[x]))
print('Final selection for ' + target_emotion, finalResult)

# show the result
finalResult = sorted(finalResult, key=lambda x:x[0])
print('Final selection for ' + target_emotion, finalResult)

NameError: name 'result_AU2' is not defined

### For Test

In [ ]:
# For test
    
    
# open images
folderName = 'anger'
a = os.path.join(folderName, '2023_02_10_15_11_49_anger_0.png')
b = os.path.join(folderName, '2023_02_10_15_11_53_anger_2.png')
# for i in range(50):
# a = os.path.join(folderName, res_value_order[0][0][:-12]+'.png')
# b = os.path.join(folderName, res_value_order[i][0][:-12]+'.png')
img1 = Image.open(a)
img2 = Image.open(b)

# make sure images have same dimensions, use .resize to scale image 2 to match image 1 dimensions
# also reduce the shape by half just to save some processing power

img1_reshape = img1.resize((round(img1.size[0]*0.5), round(img1.size[1]*0.5)))
img2_reshape = img2.resize((round(img1.size[0]*0.5), round(img1.size[1]*0.5)))
# img1 (640, 480)
# img2 (640, 480)
# img1_reshape (320, 240)

# convert the images to (R,G,B) arrays
img_array1 = np.array(img1_reshape)
img_array2 = np.array(img2_reshape)

# flatten the arrays so they are 1 dimensional vectors
img_array1 = img_array1.flatten()
img_array2 = img_array2.flatten()

# divide the arrays by 255, the maximum RGB value to make sure every value is on a 0-1 scale
# img_array1 = img_array1/255
# img_array2 = img_array2/255

print(cal_cosine_distance(img_array1, img_array2))
print(cal_cosine_similarity(img_array1, img_array2))
print(a, b)

0.11000954841219102
0.889990451587809
anger/2023_02_10_15_11_49_anger_0.png anger/2023_02_10_15_11_53_anger_2.png


In [ ]:
# greedy search

    count = 1
#     print('top_name_list', top_name_list, '\n')
    finalResult = []
    finalResult.append([0, top_name_list[0]])
    
    # construct a 10 x 10 matrix
    mat_2d = []
    for i in range(100):
        mat_2d.append(result_AU2[i][2])
    ary = np.array(mat_2d).reshape(10, 10)
    
#     finalResult.append([2, top_name_list[2]])
    while count < 5:
        x_list = []
        y_list = []
#         print()
        for i in range(len(finalResult)):
            x_list.append(finalResult[i][0])

        for i in x_list:
            y_list.append(ary[i])
#         xprint(y_list)
    #     print(ary[i])
    #     print(np.argmax(ary[i]))
        sum_v = np.zeros(10)
        for i in y_list:
            sum_v += i
        for i in x_list:
            sum_v[i] = 0
        y = np.argmax(sum_v)
        finalResult.append([y, top_name_list[y]])
        count += 1
        print('x_list', x_list)
#         print(sum_v, y)
    #         np.partition(k.flatten(), -2)[-2]
    #         print(ary[x].index(max(ary[x])))
    #     print(max(ary[x]))
    print('Final selection for ' + target_emotion, finalResult)
    
    # show the result
    finalResult = sorted(finalResult, key=lambda x:x[0])
    print('Final selection for ' + target_emotion, finalResult)
    
    
    # Copy files
    sum_folderName = "image_analysis/211223Exp/Exp7/selected_files/" + target_emotion + '/'
    if not os.path.exists(sum_folderName):
        try:
            os.mkdir(sum_folderName)
        except Exception as e:
            print(e)
    import shutil
    for i in finalResult:
        temp = pd.read_csv(folderName + i[1][:-4] + '_emotion.csv')
        targetID = RCSystem.get_target(target_emotion)
        temp_result = temp.iloc[:, -8:].iloc[0, targetID]
        print(i[0], i[1][:-4]+'.png', temp_result)
        tempFileAU = i[1]
        tempFileFig = i[1][:-4]+'.png'
        tempFilePyfeat = i[1][:-4] + '_emotion.csv'
#         shutil.copy(folderNameAU + tempFileAU, sum_folderName)
#         shutil.copy(folderName + tempFileFig, sum_folderName)
#         shutil.copy(folderName + tempFilePyfeat, sum_folderName)
    